In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('energydata_complete.csv')

In [3]:
column_names = {'lights':'Lights', 
'T1': 'T_Kitchen_Area',
'RH_1': 'RH_Kitchen_area', 
'T2': 'T_Living_Room', 
'RH_2': 'RH_Living_Room',
'T3': 'T_Laundry_Room', 
'RH_3': 'RH_Laundry_Room',
'T4': 'T_Office_Room',
'RH_4': 'RH_Office_Room',
'T5': 'T_Bathroom',
'RH_5': 'RH_Bathroom', 
'T6': 'T_Outside_North', 
'RH_6': 'RH_Outside_North',
'T7': 'T_Ironing_Room', 
'RH_7': 'RH_Ironing_Room',
'T8': 'T_Teenager_Room',
'RH_8': 'RH_Teenager_Room',
'T9': 'T_Parents_Room',
'RH_9': 'RH_Parents_Room'}

In [5]:
df = df.rename(columns=column_names)

In [4]:
df['Month'] = pd.to_datetime(df['date']).dt.month
df['Hour'] = pd.to_datetime(df['date']).dt.hour

In [7]:
to_drop = ['date', 'rv1', 'rv2']

In [8]:
df_train = df.drop(to_drop, axis=1)
df_train.head()

,Appliances,Lights,T_Kitchen_Area,RH_Kitchen_area,T_Living_Room,RH_Living_Room,T_Laundry_Room,RH_Laundry_Room,T_Office_Room,RH_Office_Room,...,T_Parents_Room,RH_Parents_Room,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,Month,Hour
0,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,1,17
1,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,1,17
2,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,1,17
3,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,1,17
4,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,1,17


In [9]:
df_test =df[["rv1","rv2"]]
df_test.head()

,rv1,rv2
0,13.275433,13.275433
1,18.606195,18.606195
2,28.642668,28.642668
3,45.410389,45.410389
4,10.084097,10.084097


In [10]:
# Mean Absolute Error (MAE)
#Firstly, we normalise our dataset to a common scale using the min max scaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normalised_df = pd.DataFrame(scaler.fit_transform(df_train), columns=df_train.columns)
features_df = normalised_df.drop(columns=['Appliances'])
target = normalised_df['Appliances']

#Now, we split our dataset into the training and testing dataset. Recall that we
# had earlier segmented the features and target variables.
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features_df, target,
test_size=0.3, random_state=1)

In [12]:
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()

In [15]:
#fit the model to the training dataset
linear_model.fit(x_train, y_train)

LinearRegression()

In [16]:
#obtain predictions
predicted_values = linear_model.predict(x_test)

In [25]:
train_acc = linear_model.score(x_train, y_train) #training accuracy
test_acc = linear_model.score(x_test, y_test) #test accuracy

print('Train Accuracy', train_acc)
print('Test Accuracy', test_acc)

Train Accuracy 0.16529709655029656
Test Accuracy 0.17535281329166963


In [17]:
#MAE
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, predicted_values)
round(mae, 3)

0.049

In [18]:
#RSS
rss = np.sum(np.square(y_test - predicted_values))
round(rss, 3)

45.065

In [19]:
#RMS
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, predicted_values))
round(rmse, 3)

0.087

In [20]:
#Rsquared
from sklearn.metrics import r2_score
r2_score = r2_score(y_test, predicted_values)
round(r2_score, 3)

0.175

In [21]:
# Ridge Regression
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=0.5)
ridge_reg.fit(x_train, y_train)

Ridge(alpha=0.5)

In [23]:
# Feature Selection and Lasso Regression

from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(x_train, y_train)

#comparing the effects of regularisation
def get_weights_df(model, feat, col_name):
    #this function returns the weight of every feature
    weights = pd.Series(model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ['Features', col_name]
    weights_df[col_name].round(3)
    return weights_df
    
linear_model_weights = get_weights_df(linear_model, x_train, 'Linear_Model_Weight')
ridge_weights_df = get_weights_df(ridge_reg, x_train, 'Ridge_Weight')
lasso_weights_df = get_weights_df(lasso_reg, x_train, 'Lasso_weight')
final_weights = pd.merge(linear_model_weights, ridge_weights_df, on='Features')
final_weights = pd.merge(final_weights, lasso_weights_df, on='Features')


In [24]:
final_weights

,Features,Linear_Model_Weight,Ridge_Weight,Lasso_weight
0,RH_Living_Room,-0.415966,-0.366905,-0.000000
1,T_out,-0.252102,-0.195038,0.000000
2,T_Living_Room,-0.215909,-0.178851,0.000000
3,RH_Teenager_Room,-0.097202,-0.095778,-0.000000
4,T_Parents_Room,-0.093889,-0.092111,-0.000000
5,RH_Office_Room,-0.047230,-0.049097,0.000000
6,RH_out,-0.040485,-0.018991,-0.024379
7,Month,-0.040311,-0.041987,-0.000000
8,T_Office_Room,-0.039661,-0.040779,-0.000000
9,RH_Ironing_Room,-0.037983,-0.039723,-0.000000
